In [25]:
import pandas as pd 
import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
from pprint import pprint
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

In [16]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [19]:
data_lemmatized = list(pd.read_csv('../data/authors_with_biography_lemmatized.csv')["biography_cleaned"])
data_lemmatized = list(sent_to_words(data_lemmatized))
data_lemmatized[2]

['kansas_city',
 'missouri',
 'avocat',
 'mere',
 'peintr',
 'revele',
 'tr',
 'jeune',
 'doue',
 'dessin',
 'debut',
 'caricaturiste',
 'star',
 'journal',
 'post',
 'abandonn',
 'etud',
 'chicago',
 'tenter',
 'linstitut',
 'dart',
 'chicago',
 'renonc',
 'vivre',
 'ville',
 'revenir',
 'remarier',
 'jenning',
 'premier',
 'epous',
 'commencer',
 'contribuer',
 'magazine',
 'satirique',
 'puck',
 'demenage',
 'femme',
 'fille',
 'poursuivre',
 'travail',
 'puck',
 'mccall',
 'louer',
 'ensuite',
 'atelier',
 'quil',
 'partager',
 'peintr',
 'benton',
 'cest',
 'servir',
 'premier',
 'modele',
 'caricaturer',
 'aller',
 'connaitre',
 'certain',
 'puck',
 'decide',
 'denvoyer',
 'mission',
 'reportage',
 'guerre',
 'sattach',
 'pays',
 'ville',
 'lumiere',
 'vie',
 'frequent',
 'sejour',
 'juin',
 'nommer',
 'chevalier',
 'titrer',
 'etranger',
 'qualite',
 'quartist',
 'dessinateur',
 'reussit',
 'croquer',
 'plupart',
 'celebrit',
 'temps',
 'dernier',
 'croquis',
 'charlie',
 'chapl

In [21]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:2])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 2), (42, 1), (43, 1), (44, 1), (45, 1), (46, 2), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1)], [(60, 1), (61, 1), (62, 2), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 2), (87, 2), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1)]]


In [22]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('aarts', 1),
  ('acheve', 1),
  ('alfaguara', 1),
  ('an', 1),
  ('apr', 1),
  ('artistique', 1),
  ('beaux_art', 1),
  ('bourse', 1),
  ('braga', 1),
  ('camiler', 1),
  ('carrasco', 1),
  ('carthagene', 1),
  ('chil', 1),
  ('chile', 1),
  ('clemencia', 1),
  ('colegio', 1),
  ('debut', 1),
  ('depression', 1),
  ('enfance', 1),
  ('enfant', 1),
  ('enterprend', 1),
  ('estamper', 1),
  ('etud', 1),
  ('etude', 1),
  ('etudier', 1),
  ('famille', 1),
  ('fernandez', 1),
  ('fille', 1),
  ('fort', 1),
  ('frere', 1),
  ('habiter', 1),
  ('ignacio', 1),
  ('illustrateur', 1),
  ('jusqua', 1),
  ('lage', 1),
  ('lainer', 1),
  ('leagu', 1),
  ('lecole', 1),
  ('lecriture', 1),
  ('lescuela', 1),
  ('livr', 1),
  ('llay', 2),
  ('luniversidad', 1),
  ('marta', 1),
  ('nee', 1),
  ('new_york', 1),
  ('passer', 2),
  ('petit', 1),
  ('poursuivre', 1),
  ('premier', 1),
  ('province', 1),
  ('publier', 1),
  ('rioseco', 1),
  ('saint', 1),
  ('scolarite', 1),
  ('sinstaller', 1),
  ('stu

In [23]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=33,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [26]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.011*"maitre" + 0.010*"militaire" + 0.006*"tsvetaieva" + 0.006*"tableau" + '
  '0.005*"grec" + 0.005*"navir" + 0.005*"mathematiqu" + 0.005*"voyager" + '
  '0.005*"assassin" + 0.005*"armee"'),
 (1,
  '0.010*"mourir" + 0.010*"alors" + 0.007*"coder" + 0.007*"prendre" + '
  '0.006*"quil" + 0.006*"nouveau" + 0.006*"bien" + 0.005*"voir" + 0.005*"apr" '
  '+ 0.005*"tout"'),
 (2,
  '0.021*"an" + 0.013*"premier" + 0.010*"apr" + 0.010*"grand" + 0.009*"mourir" '
  '+ 0.008*"egalement" + 0.007*"janvier" + 0.006*"fille" + 0.006*"vie" + '
  '0.005*"ecrit"'),
 (3,
  '0.039*"film" + 0.015*"domaine" + 0.008*"realisateur" + 0.007*"site" + '
  '0.006*"cinemer" + 0.006*"role" + 0.005*"serie" + 0.005*"apparer" + '
  '0.005*"poisson" + 0.005*"ministere"'),
 (4,
  '0.010*"collection" + 0.008*"luniversite" + 0.007*"bibliothequ" + '
  '0.007*"ouvrager" + 0.006*"edition" + 0.006*"souffrir" + 0.005*"litterature" '
  '+ 0.004*"remporter" + 0.003*"lassociation" + 0.003*"tokyo"'),
 (5,
  '0.011*"coder" + 0

In [27]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

...

C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(


Perplexity:  -10.10222342017409


C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

NameError: name 'CoherenceModel' is not defined

In [28]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\mateg\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.303324 -0.086086       1        1  32.007774
2      0.244532  0.099312       2        1  18.890768
5      0.127152 -0.120712       3        1  13.454664
8      0.000745  0.242962       4        1  10.011014
6     -0.100562  0.111373       5        1   7.484435
0     -0.118951 -0.033037       6        1   4.789840
4     -0.109963 -0.056854       7        1   3.840375
3     -0.104408 -0.058352       8        1   3.417426
7     -0.121487 -0.045576       9        1   3.123061
9     -0.120381 -0.053030      10        1   2.980635, topic_info=      Category         Freq            Term        Total  loglift  logprob
531    Default   636.000000             nan   636.000000  30.0000  30.0000
3      Default  1480.000000              an  1480.000000  29.0000  29.0000
590    Default   430.000000            film   430.000000  28.0000  28.0000
15952  Default   352.000000            clan   352.000000  27.0000  27.0000
49     Default  1305.000000         premier  1305.000000  26.0000  26.0000
78     Default  1586.000000          mourir  1586.000000  25.0000  25.0000
179    Default   592.000000          guerre   592.000000  24.0000  24.0000
1306   Default   388.000000         general   388.000000  23.0000  23.0000
220    Default   398.000000          nommer   398.000000  22.0000  22.0000
449    Default  1262.000000           alors  1262.000000  21.0000  21.0000
597    Default   351.000000         juillet   351.000000  20.0000  20.0000
9183   Default   327.000000       batailler   327.000000  19.0000  19.0000
103    Default   366.000000        allemand   366.000000  18.0000  18.0000
456    Default   376.000000            chef   376.000000  17.0000  17.0000
379    Default  1124.000000           grand  1124.000000  16.0000  16.0000
1384   Default   351.000000       militaire   351.000000  15.0000  15.0000
62     Default  1492.000000           coder  1492.000000  14.0000  14.0000
4      Default  1479.000000             apr  1479.000000  13.0000  13.0000
1379   Default   404.000000         janvier   404.000000  12.0000  12.0000
680    Default   455.000000           avril   455.000000  11.0000  11.0000
489    Default   311.000000     luniversite   311.000000  10.0000  10.0000
42970  Default   185.000000       hideyoshi   185.000000   9.0000   9.0000
311    Default   595.000000            lors   595.000000   8.0000   8.0000
161    Default   700.000000       egalement   700.000000   7.0000   7.0000
3723   Default   304.000000         italien   304.000000   6.0000   6.0000
262    Default   408.000000       septembre   408.000000   5.0000   5.0000
280    Default   298.000000          titrer   298.000000   4.0000   4.0000
809    Default   416.000000          membre   416.000000   3.0000   3.0000
87     Default   669.000000            quil   669.000000   2.0000   2.0000
50     Default   185.000000        province   185.000000   1.0000   1.0000
...        ...          ...             ...          ...      ...      ...
208    Topic10    30.753933        magazine    31.620863   3.4852  -5.7473
36820  Topic10    31.789955       chantilly    32.689560   3.4851  -5.7142
1587   Topic10    29.684643        domicile    30.551708   3.4842  -5.7827
936    Topic10    28.511517        cuisiner    29.378719   3.4831  -5.8230
13539  Topic10    23.567621          street    24.434875   3.4769  -6.0135
25521  Topic10    22.201426        appareil    23.068335   3.4747  -6.0732
31599  Topic10    22.199648           radar    23.067842   3.4747  -6.0732
19712  Topic10    20.155176       laviation    21.022051   3.4709  -6.1699
8697   Topic10    20.069693        reporter    20.936897   3.4707  -6.1741
16392  Topic10    20.005901   fonctionnaire    20.874567   3.4705  -6.1773
11786  Topic10    19.669758          profil    20.538958   3.4698  -6.1942
2118   Topic10    19.077394  chambre_dhotel    19.944807   3.4686  -6.2248
5770   To

In [29]:
pyLDAvis.save_html(vis, 'lda.html')